<a href="https://colab.research.google.com/github/gowthamsaib/solved-problems/blob/main/Copy_of_fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# -------------------------
# 3) PREPROCESS & SPLIT
# -------------------------
# One-hot encode categoricals
dataset = pd.get_dummies(dataset, drop_first=True)

# 80/20 train/test
from sklearn.model_selection import train_test_split
train_dataset, test_dataset = train_test_split(
    dataset, test_size=0.2, random_state=42
)

# Separate out the labels
train_labels = train_dataset.pop('expenses')
test_labels  = test_dataset .pop('expenses')

# -------------------------
# 4) NORMALIZATION LAYER
# -------------------------
# Specify input_shape so downstream layers build immediately
normalizer = layers.Normalization(
    axis=-1,
    input_shape=[train_dataset.shape[1]]
)
normalizer.adapt(np.array(train_dataset))

# -------------------------
# 5) BUILD & COMPILE MODEL (IMPROVED)
# -------------------------
from tensorflow.keras.losses import Huber

def build_and_compile_model():
    m = keras.Sequential([
        normalizer,
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1),
    ])
    m.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=Huber(),                # more robust to outliers
        metrics=['mae', 'mse']
    )
    return m

model = build_and_compile_model()
# Force-build so summary shows all params
model.build(input_shape=(None, train_dataset.shape[1]))
model.summary()

# -------------------------
# 6) TRAIN (EXTENDED)
# -------------------------
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,            # wait longer for improvement
    restore_best_weights=True
)

history = model.fit(
    train_dataset,
    train_labels,
    validation_split=0.2,
    epochs=125,             # allow more training
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
